# Non-tradable Bonds by Date

Filter bonds with missing/invalid TWAP prices for a given date.

In [5]:
import re
from pathlib import Path
import pandas as pd

# Optional: set repo root manually
# repo_root = Path('C:/Users/BaiYang/CBOND_DAY')
repo_root = None

root = Path.cwd()
if repo_root is None:
    while root != root.parent and not (root / 'cbond_daily').exists():
        root = root.parent
    if not (root / 'cbond_daily').exists():
        raise FileNotFoundError('Cannot find repo root containing cbond_daily. Set repo_root explicitly.')
else:
    root = repo_root

# Load paths_config.json5 (minimal parsing)
paths_cfg = root / 'cbond_daily' / 'config' / 'paths_config.json5'
text = paths_cfg.read_text(encoding='utf-8')

m = re.search(r'dwd_root\s*:\s*"([^"]+)"', text)
if not m:
    raise ValueError('Cannot parse dwd_root from paths_config.json5')

dwd_root = Path(m.group(1))
print('dwd_root:', dwd_root)


dwd_root: D:\cbond_daily\clean_data


In [6]:
# Settings
view_date = '20260112'  # YYYYMMDD
buy_twap_col = 'twap_0945_1000'
sell_twap_col = 'twap_1430_1442'


In [7]:
# Load daily cleaned data
from datetime import datetime

trade_day = datetime.strptime(view_date, '%Y%m%d').date()
month = f"{trade_day.year:04d}-{trade_day.month:02d}"
path = dwd_root / month / f"{view_date}.parquet"
if not path.exists():
    raise FileNotFoundError(f'cleaned data not found: {path}')

df = pd.read_parquet(path)
print('rows:', len(df))

required = ['code', buy_twap_col, sell_twap_col]
missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f'missing columns: {missing}')

tradable_mask = (
    df[buy_twap_col].notna()
    & df[sell_twap_col].notna()
    & (df[buy_twap_col] > 0)
    & (df[sell_twap_col] > 0)
)

non_tradable = df.loc[~tradable_mask, ['code', buy_twap_col, sell_twap_col]]
print('non_tradable count:', len(non_tradable))
non_tradable.head()


rows: 638
non_tradable count: 258


,code,twap_0945_1000,twap_1430_1442
21,110148.SZ,NaN,NaN
22,110150.SZ,NaN,NaN
23,110152.SZ,NaN,NaN
24,110154.SZ,NaN,NaN
25,110159.SZ,NaN,NaN


## Preview Non-tradable Codes

In [8]:
# Preview non-tradable codes in notebook
preview = non_tradable[['code']].drop_duplicates().reset_index(drop=True)
print('non_tradable unique codes:', len(preview))
preview.head(50)


non_tradable unique codes: 258


,code
0,110148.SZ
1,110150.SZ
2,110152.SZ
3,110154.SZ
4,110159.SZ
5,110165.SZ
6,110169.SZ
7,110170.SZ
8,110171.SZ
9,110172.SZ
